# Lume: How to auto-map data in your code

After running a job to create a mapper, you can use it as an embedded pipeline in your code. This is especially helpful to create pipelines where similar data will come in, and you need to embed a static mapper after creating it with Lume.

### Overview

This notebook contains the following 1 section:

- **Map incoming source data using an existing pipeline:** Specify a set of functions and use the Lume API to map data.

## Map incoming source data using an existing pipeline

### Utilities

First let's define a few utilities for making calls to the Lume API.

In [ ]:
%pip install httpx

In [24]:
import httpx 
import traceback
url = "https://api.lume.ai"
api_key = '<YOUR_API_KEY'

In [3]:
async def get_pipeline(pipeline_id):
    new_url = f'{url}/pipelines/{pipeline_id}'
    headers = {"lume-api-key": api_key}
    async with httpx.AsyncClient(timeout=60) as client:
        job = await client.get(new_url, headers=headers)
        job = job.json()
    return job

In [4]:
async def create_job(pipeline_id, data):
    new_url = f'{url}/pipelines/{pipeline_id}/jobs'
    headers = {"lume-api-key": api_key}
    payload = {
        "data": data
    }
    async with httpx.AsyncClient(timeout=60) as client:
        job = await client.post(new_url, headers=headers, json=payload)
        job = job.json()
    return job

In [5]:
async def run_job(job_id):
    new_url = f'{url}/jobs/{job_id}/run'
    headers = {"lume-api-key": api_key}
    
    async with httpx.AsyncClient(timeout=6000) as client:
        
        job = await client.post(new_url, headers=headers)
        job = job.json()
    return job

In [6]:
async def get_result(result_id):
    new_url = f'{url}/results/{result_id}'
    headers = {"lume-api-key": api_key}
    async with httpx.AsyncClient(timeout=60) as client:
        job = await client.get(new_url, headers=headers)
        job = job.json()
    return job

In [7]:
async def get_mappings_from_result(result_id, page=1, size=50):
    new_url = f'{url}/results/{result_id}/mappings'
    headers = {"lume-api-key": api_key}
    params = {
        'page': page,  # Assuming you want to access the second page
        'size': size  # Number of records per page
    }
    async with httpx.AsyncClient(timeout=60) as client:
        job = await client.get(new_url, headers=headers, params=params)
        job = job.json()
    return job 

In [8]:
async def get_pipeline_with_name(pipeline_name):
    new_url = f'{url}/pipelines'
    headers = {"lume-api-key": api_key}
    async with httpx.AsyncClient(timeout=60) as client:
        ret = await client.get(new_url, headers=headers)
        pipelinePage = ret.json()
    pipelines = pipelinePage['items'] #Paginated return. To access the page's data, use ['items']. 
    for pipeline in pipelines:
        if pipeline['name'] == pipeline_name:
            return pipeline
    return None

In [9]:
async def get_all_mappings(result_id):
    mappings = []
    first_page = await get_mappings_from_result(result_id)
    mappings.extend(first_page['items'])

    total_items = first_page['total']
    page_size = first_page['size']
    total_pages = total_items // page_size + 1

    for page in range(2, total_pages + 1):
        new_mappings_page = await get_mappings_from_result(result_id, page=page)
        mappings.extend(new_mappings_page['items'])
    return mappings

In [10]:
async def create_transformation_job(pipeline, source):
    try:
        job = await create_job(pipeline['id'], source)
        result = await run_job(job['id'])
        result = await get_mappings_from_result(result['id'])
        return result
    except Exception as e:
        print("ERROR")
        print(e)
        print(traceback.format_exc())
        return

### Prior Context

This cookbook assumes a pipeline has already been created, called `ecomm_test`. The existing pipeline is meant to map source ecommerce data to an internal ecommerce data model. The target schema used in the pipeline is below.

In [ ]:
{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "type": "object",
  "properties": {
    "product": {
      "type": "object",
      "properties": {
        "product-make-model": {
          "type": "string",
          "description": "The make, model, and version of the product. Should be formatted by Make, Model, Version"
        },
        "body_html": {
          "type": "string",
          "description": "HTML content describing the product."
        },
        "created_at": {
          "type": "string",
          "format": "date-time",
          "description": "The date and time when the product was created."
        },
        "handle": {
          "type": "string",
          "description": "A unique string identifier for the product."
        },
        "id": {
          "type": "integer",
          "description": "The unique numeric identifier for the product."
        },
        "currency": {
          "type": [
            "string",
            "None"
          ],
          "description": "Choose a currency to assign to the product prices."
        },
        "description": {
          "type": [
            "string",
            "None"
          ],
          "description": "Product description (plaintext, read-only)."
        },
        "name": {
          "type": [
            "string",
            "None"
          ],
          "description": "Product name."
        },
        "weight": {
          "type": [
            "number",
            "null"
          ],
          "description": "Weight of the Product."
        },
        "sku": {
          "type": [
            "string",
            "null"
          ],
          "description": "Stock keeping unit of a product. Note: this field is only used if a product doesn't use variants."
        },
        "cost_price": {
          "type": [
            "string",
            "null"
          ],
          "description": "Default cost price for the product."
        },
        "product_type": {
          "type": [
            "string",
            "null"
          ],
          "description": "ID of the type that product belongs to."
        },
        "seller": {
          "type": [
            "string",
            "null"
          ],
          "description": "ID of the seller that product belongs to."
        },
        "image_url": {
          "type": [
            "string",
            "null"
          ],
          "description": "URL of the product image."
        },
        "published_at": {
          "type": "string",
          "format": "date-time",
          "description": "The date and time when the product was published, in ISO 8601 format."
        },
        "published_scope": {
          "type": "string",
          "enum": [
            "global",
            "web",
            "point_of_sale"
          ],
          "description": "The scope in which the product is published. Can be 'global', 'web', or 'point_of_sale'."
        },
        "status": {
          "type": "string",
          "enum": [
            "active",
            "archive",
            "draft"
          ],
          "description": "The current status of the product."
        },
        "tags": {
          "type": "string",
          "description": "Comma-separated tags associated with the product."
        },
        "template_suffix": {
          "type": "string",
          "description": "A custom template suffix to be used for displaying the product. Can be 'null' if no custom template is used."
        },
        "title": {
          "type": "string",
          "description": "The title of the product."
        },
        "updated_at": {
          "type": "string",
          "format": "date-time",
          "description": "The last date and time when the product information was updated, in ISO 8601 format."
        },
        "image": {
          "type": "object",
          "properties": {
            "id": {
              "type": "integer",
              "description": "The unique numeric identifier for the image."
            },
            "created_at": {
              "type": "string",
              "format": "date-time",
              "description": "The date and time when the image was created."
            },
            "updated_at": {
              "type": "string",
              "format": "date-time",
              "description": "The date and time when the image was last updated."
            },
            "width": {
              "type": "integer",
              "description": "The width of the image in pixels."
            },
            "height": {
              "type": "integer",
              "description": "The height of the image in pixels."
            },
            "x": {
              "type": "string",
              "format": "uri",
              "description": "The URL of the image."
            }
          },
          "required": [
            "id",
            "created_at",
            "updated_at",
            "width",
            "height",
            "x"
          ]
        }
      },
      "required": [
        "body_html",
        "created_at",
        "handle",
        "id",
        "currency",
        "description",
        "name",
        "weight",
        "sku",
        "cost_price",
        "product_type",
        "seller",
        "image_url",
        "published_at",
        "published_scope",
        "status",
        "title",
        "updated_at",
        "image"
      ]
    },
    "metadata": {
      "type": "object",
      "description": "The metadata of the product",
      "properties": {
        "vendor_address": {
          "type": "object",
          "description": "The address details of the vendor",
          "properties": {
            "city": {
              "type": [
                "string",
                "null"
              ],
              "description": "City of the vendor address."
            },
            "country": {
              "type": [
                "string",
                "null"
              ],
              "description": "Country of the vendor address."
            },
            "line1": {
              "type": [
                "string",
                "null"
              ],
              "description": "Line 1 of the vendor address."
            },
            "line2": {
              "type": [
                "string",
                "null"
              ],
              "description": "Line 2 of the vendor address."
            },
            "postal_code": {
              "type": [
                "string",
                "null"
              ],
              "description": "Postal code of the vendor address."
            },
            "state": {
              "type": [
                "string",
                "null"
              ],
              "description": "State of the vendor address."
            }
          }
        },
        "transaction_full_address": {
          "type": "string",
          "description": "The full address details of one transaction"
        }
      }
    },
    "statistics": {
      "type": "object",
      "description": "The statistics of the product",
      "properties": {
        "total_sales": {
          "type": [
            "number",
            "null"
          ],
          "description": "Total sales of the product measured by transactions"
        },
        "largest_transaction_delta": {
          "type": [
            "number",
            "null"
          ],
          "description": "The largest delta in transaction total of the product, from the most expensive transaction and the cheapest transaction"
        },
        "largest_transaction_cost": {
          "type": [
            "number",
            "null"
          ],
          "description": "The largest transaction cost of the product."
        },
        "lowest_transaction_cost": {
          "type": [
            "number",
            "null"
          ],
          "description": "The lowest transaction cost of the product."
        },
        "average_transaction_cost": {
          "type": [
            "number",
            "null"
          ],
          "description": "The average transaction cost of the product."
        }
      }
    },
    "reviews": {
      "type": [
        "object",
        "null"
      ],
      "items": {
        "type": "object",
        "properties": {
          "review_id": {
            "type": "integer",
            "description": "Unique identifier for the review."
          },
          "author": {
            "type": "string",
            "description": "The author of the review."
          },
          "rating": {
            "type": "integer",
            "description": "The rating given in the review."
          },
          "comment": {
            "type": "string",
            "description": "The text of the review."
          }
        },
        "required": [
          "review_id",
          "author",
          "rating"
        ]
      },
      "description": "Customer reviews of the product."
    }
  },
  "required": [
    "product",
    "metadata",
    "statistics",
    "reviews"
  ]
}

### Getting Started

Let's access our source data and use a Lume pipeline to map it automatically.

Below is the data arriving from your source.

In [12]:
source_data = [
    {
      "product": {
        "make": "iPod",
        "model": "Nano",
        "version": "V2.3",
        "body_html": "It's the small iPod with a big idea: Video.",
        "created_at": "2012-02-15T15:12:21-05:00",
        "handle": "ipod-nano",
        "id": 632910392,
        "image": {
          "id": 850703190,
          "created_at": "2018-01-08T12:34:47-05:00",
          "updated_at": "2018-01-08T12:34:47-05:00",
          "width": 110,
          "height": 140,
          "x": "http://example.com/burton.jpg"
        },
        "options": {
          "id": 594680422,
          "product_id": 632910392,
          "name": "Color",
          "position": 1,
          "values": [
            "Pink",
            "Red",
            "Green",
            "Black"
          ]
        },
        "product_type": "Cult Products",
        "published_at": "2007-12-31T19:00:00-05:00",
        "published_scope": "global",
        "status": "active",
        "tags": "Emotive, Flash Memory, MP3, Music",
        "template_suffix": "special",
        "title": "IPod Nano - 8GB",
        "updated_at": "2012-08-24T14:01:47-04:00",
        "variants": [
          {
            "barcode": "1234_pink",
            "compare_at_price": None,
            "created_at": "2012-08-24T14:01:47-04:00",
            "fulfillment_service": "manual",
            "grams": 567,
            "weight": 0.2,
            "weight_unit": "kg",
            "id": 808950810,
            "inventory_item_id": 341629,
            "inventory_management": "shopify",
            "inventory_policy": "continue",
            "inventory_quantity": 10,
            "option1": "Pink",
            "position": 1,
            "price": 199.99,
            "product_id": 632910392,
            "requires_shipping": True,
            "sku": "IPOD2008PINK",
            "taxable": True,
            "title": "Pink",
            "updated_at": "2012-08-24T14:01:47-04:00"
          },
          {
            "barcode": "5678_blue",
            "compare_at_price": None,
            "created_at": "2012-08-24T14:01:47-04:00",
            "fulfillment_service": "manual",
            "grams": 567,
            "weight": 0.2,
            "weight_unit": "kg",
            "id": 808950811,
            "inventory_item_id": 341630,
            "inventory_management": "shopify",
            "inventory_policy": "continue",
            "inventory_quantity": 15,
            "option1": "Blue",
            "position": 2,
            "price": 199.99,
            "product_id": 632910392,
            "requires_shipping": True,
            "sku": "IPOD2008BLUE",
            "taxable": True,
            "title": "Blue",
            "updated_at": "2012-08-24T14:01:47-04:00"
          },
          {
            "barcode": "9012_silver",
            "compare_at_price": None,
            "created_at": "2012-08-24T14:01:47-04:00",
            "fulfillment_service": "manual",
            "grams": 567,
            "weight": 0.2,
            "weight_unit": "kg",
            "id": 808950812,
            "inventory_item_id": 341631,
            "inventory_management": "shopify",
            "inventory_policy": "continue",
            "inventory_quantity": 20,
            "option1": "Silver",
            "position": 3,
            "price": 199.99,
            "product_id": 632910392,
            "requires_shipping": True,
            "sku": "IPOD2008SILVER",
            "taxable": True,
            "title": "Silver",
            "updated_at": "2012-08-24T14:01:47-04:00"
          }
        ],
        "vendor": "Apple",
        "vendor_full_mailing_address": "1 infinite loop, Cupertino, CA 95014 USA"
      },
      "recent_transactions": {
        "transaction_1": {
          "billing_address": {
            "address1": "2259 Park Ct",
            "address2": "Apartment 5",
            "city": "Drayton Valley",
            "company": None,
            "country": "Canada",
            "first_name": "Christopher",
            "last_name": "Gorski",
            "phone": "(555)555-5555",
            "province": "Alberta",
            "zip": "T0E 0M0",
            "name": "Christopher Gorski",
            "province_code": "AB",
            "country_code": "CA",
            "latitude": "45.41634",
            "longitude": "-75.6868"
          },
          "confirmation_number": "TLQDA3NN9",
          "currency": "USD",
          "customer": {
            "id": 207119551,
            "email": "bob.norman@mail.example.com",
            "accepts_marketing": False,
            "created_at": "2012-03-13T16:09:55-04:00",
            "updated_at": "2012-03-13T16:09:55-04:00",
            "first_name": "Bob",
            "last_name": "Norman",
            "state": "disabled",
            "note": None,
            "verified_email": True,
            "multipass_identifier": None,
            "tax_exempt": False,
            "tax_exemptions": {},
            "phone": "+13125551212",
            "tags": "loyal",
            "currency": "USD",
            "addresses": {},
            "admin_graphql_api_id": "gid://shopify/Customer/207119551",
            "default_address": {}
          },
          "id": 450789469,
          "order_number": 1001,
          "processed_at": "2008-01-10T11:00:00-05:00",
          "subtotal_price": "398.00",
          "total_tax": "11.94",
          "total_tip_received": "4.87",
          "total_weight": 300,
          "updated_at": "2012-08-24T14:02:15-04:00",
          "user_id": 31522279,
          "order_status_url": {
            "order_status_url": "https://checkout.shopify.com/112233/checkouts/4207896aad57dfb159/thank_you_token?key=753621327b9e8a64789651bf221dfe35"
          }
        },
        "transaction_2": {
          "billing_address": {
            "address1": "123 Main St",
            "address2": "Suite 10",
            "city": "Los Angeles",
            "company": "ABC Inc.",
            "country": "USA",
            "first_name": "Jennifer",
            "last_name": "Smith",
            "phone": "(555)123-4567",
            "province": "California",
            "zip": "90001",
            "name": "Jennifer Smith",
            "province_code": "CA",
            "country_code": "US",
            "latitude": "34.0522",
            "longitude": "-118.2437"
          },
          "confirmation_number": "KJASD98QW",
          "currency": "USD",
          "customer": {
            "id": 987654321,
            "email": "jennifer.smith@example.com",
            "accepts_marketing": True,
            "created_at": "2020-05-20T09:30:45-07:00",
            "updated_at": "2020-05-22T14:45:22-07:00",
            "first_name": "Jennifer",
            "last_name": "Smith",
            "state": "active",
            "note": "Preferred customer",
            "verified_email": True,
            "multipass_identifier": None,
            "tax_exempt": False,
            "tax_exemptions": {},
            "phone": "+15551234567",
            "tags": "loyal, preferred",
            "currency": "USD",
            "addresses": {},
            "admin_graphql_api_id": "gid://shopify/Customer/987654321",
            "default_address": {}
          },
          "id": 987654321,
          "order_number": 1002,
          "processed_at": "2024-02-03T15:30:00-08:00",
          "subtotal_price": "299.99",
          "total_tax": "9.00",
          "total_tip_received": "3.50",
          "total_weight": 250,
          "updated_at": "2024-02-03T16:45:00-08:00",
          "user_id": 654321987,
          "order_status_url": {
            "order_status_url": "https://checkout.shopify.com/112233/checkouts/987654321qwertyuiop/token?key=1234567890abcdef"
          }
        },
        "transaction_3": {
          "billing_address": {
            "address1": "456 Elm Street",
            "address2": "Apt 3B",
            "city": "Chicago",
            "company": "XYZ Corp",
            "country": "USA",
            "first_name": "Sarah",
            "last_name": "Johnson",
            "phone": "(555)789-0123",
            "province": "Illinois",
            "zip": "60601",
            "name": "Sarah Johnson",
            "province_code": "IL",
            "country_code": "US",
            "latitude": "41.8781",
            "longitude": "-87.6298"
          },
          "confirmation_number": "ASD12DFG6",
          "currency": "USD",
          "customer": {
            "id": 123456789,
            "email": "sarah.johnson@example.com",
            "accepts_marketing": True,
            "created_at": "2023-09-15T12:20:30-05:00",
            "updated_at": "2023-09-15T12:20:30-05:00",
            "first_name": "Sarah",
            "last_name": "Johnson",
            "state": "active",
            "note": "Frequent shopper",
            "verified_email": True,
            "multipass_identifier": None,
            "tax_exempt": False,
            "tax_exemptions": {},
            "phone": "+15557890123",
            "tags": "loyal, frequent",
            "currency": "USD",
            "addresses": {},
            "admin_graphql_api_id": "gid://shopify/Customer/123456789",
            "default_address": {}
          },
          "id": 789123456,
          "order_number": 1003,
          "processed_at": "2024-01-25T10:15:00-06:00",
          "subtotal_price": "179.99",
          "total_tax": "5.40",
          "total_tip_received": "2.25",
          "total_weight": 150,
          "updated_at": "2024-01-25T11:30:00-06:00",
          "user_id": 987654321,
          "order_status_url": {
            "order_status_url": "https://checkout.shopify.com/112233/checkouts/789123456qwertyuiop/token?key=0987654321abcdefg"
          }
        }
      }
    },
    {
      "product": {
        "make": "Apple",
        "model": "MacBook Pro",
        "version": "2024",
        "body_html": "Experience the power of the MacBook Pro.",
        "created_at": "2024-02-05T10:30:00-08:00",
        "handle": "macbook-pro",
        "id": 9876543210,
        "image": {
          "id": 999888777,
          "created_at": "2024-02-05T10:45:00-08:00",
          "updated_at": "2024-02-05T10:45:00-08:00",
          "width": 150,
          "height": 200,
          "x": "http://example.com/macbook-pro.jpg"
        },
        "options": {
          "id": 1234567890,
          "product_id": 9876543210,
          "name": "Color",
          "position": 1,
          "values": [
            "Space Gray",
            "Silver",
            "Rose Gold"
          ]
        },
        "product_type": "Laptops",
        "published_at": "2024-02-05T11:00:00-08:00",
        "published_scope": "global",
        "status": "active",
        "tags": "Powerful, High-Performance, MacBook",
        "template_suffix": "special",
        "title": "MacBook Pro - 16-inch",
        "updated_at": "2024-02-05T10:45:00-08:00",
        "variants": [
          {
            "barcode": "1234_spacegray",
            "compare_at_price": None,
            "created_at": "2024-02-05T10:45:00-08:00",
            "fulfillment_service": "manual",
            "grams": 1800,
            "weight": 1.8,
            "weight_unit": "kg",
            "id": 9876543211,
            "inventory_item_id": 4567890123,
            "inventory_management": "shopify",
            "inventory_policy": "continue",
            "inventory_quantity": 25,
            "option1": "Space Gray",
            "position": 1,
            "price": 2199.99,
            "product_id": 9876543210,
            "requires_shipping": True,
            "sku": "MBP2024SG",
            "taxable": True,
            "title": "Space Gray",
            "updated_at": "2024-02-05T10:45:00-08:00"
          },
          {
            "barcode": "5678_silver",
            "compare_at_price": None,
            "created_at": "2024-02-05T10:45:00-08:00",
            "fulfillment_service": "manual",
            "grams": 1800,
            "weight": 1.8,
            "weight_unit": "kg",
            "id": 9876543212,
            "inventory_item_id": 4567890124,
            "inventory_management": "shopify",
            "inventory_policy": "continue",
            "inventory_quantity": 20,
            "option1": "Silver",
            "position": 2,
            "price": 2199.99,
            "product_id": 9876543210,
            "requires_shipping": True,
            "sku": "MBP2024SILVER",
            "taxable": True,
            "title": "Silver",
            "updated_at": "2024-02-05T10:45:00-08:00"
          },
          {
            "barcode": "9012_rosegold",
            "compare_at_price": None,
            "created_at": "2024-02-05T10:45:00-08:00",
            "fulfillment_service": "manual",
            "grams": 1800,
            "weight": 1.8,
            "weight_unit": "kg",
            "id": 9876543213,
            "inventory_item_id": 4567890125,
            "inventory_management": "shopify",
            "inventory_policy": "continue",
            "inventory_quantity": 15,
            "option1": "Rose Gold",
            "position": 3,
            "price": 2199.99,
            "product_id": 9876543210,
            "requires_shipping": True,
            "sku": "MBP2024ROSEGOLD",
            "taxable": True,
            "title": "Rose Gold",
            "updated_at": "2024-02-05T10:45:00-08:00"
          }
        ],
        "vendor": "Apple Inc.",
        "vendor_full_mailing_address": "1 Infinite Loop, Cupertino, CA 95014 USA"
      },
      "recent_transactions": {
        "transaction_3": {
          "billing_address": {
            "address1": "789 Oak Lane",
            "address2": "Suite 15A",
            "city": "New York",
            "company": "Tech Solutions Inc.",
            "country": "USA",
            "first_name": "Michael",
            "last_name": "Anderson",
            "phone": "(555)789-1234",
            "province": "New York",
            "zip": "10001",
            "name": "Michael Anderson",
            "province_code": "NY",
            "country_code": "US",
            "latitude": "40.7128",
            "longitude": "-74.0060"
          },
          "confirmation_number": "QWERTYUIO1",
          "currency": "USD",
          "customer": {
            "id": 456789012,
            "email": "michael.anderson@example.com",
            "accepts_marketing": True,
            "created_at": "2023-11-05T14:45:30-05:00",
            "updated_at": "2023-11-05T14:45:30-05:00",
            "first_name": "Michael",
            "last_name": "Anderson",
            "state": "active",
            "note": "VIP customer",
            "verified_email": True,
            "multipass_identifier": None,
            "tax_exempt": False,
            "tax_exemptions": {},
            "phone": "+15557891234",
            "tags": "loyal, VIP",
            "currency": "USD",
            "addresses": {},
            "admin_graphql_api_id": "gid://shopify/Customer/456789012",
            "default_address": {}
          },
          "id": 1234567890,
          "order_number": 1004,
          "processed_at": "2024-02-15T09:30:00-05:00",
          "subtotal_price": "2499.99",
          "total_tax": "74.97",
          "total_tip_received": "9.50",
          "total_weight": 3000,
          "updated_at": "2024-02-15T10:45:00-05:00",
          "user_id": 789012345,
          "order_status_url": {
            "order_status_url": "https://checkout.shopify.com/112233/checkouts/1234567890qwertyuiop/token?key=abcdefgh1234567890"
          }
        },
        "transaction_2": {
          "billing_address": {
            "address1": "456 Pine Street",
            "address2": "Apt 4C",
            "city": "San Francisco",
            "company": "Tech Innovations LLC",
            "country": "USA",
            "first_name": "Emily",
            "last_name": "Johnson",
            "phone": "(555)234-5678",
            "province": "California",
            "zip": "94101",
            "name": "Emily Johnson",
            "province_code": "CA",
            "country_code": "US",
            "latitude": "37.7749",
            "longitude": "-122.4194"
          },
          "confirmation_number": "ZXCVBNM2",
          "currency": "USD",
          "customer": {
            "id": 567890123,
            "email": "emily.johnson@example.com",
            "accepts_marketing": True,
            "created_at": "2022-07-10T10:15:00-07:00",
            "updated_at": "2022-07-12T15:30:45-07:00",
            "first_name": "Emily",
            "last_name": "Johnson",
            "state": "active",
            "note": "Returning customer",
            "verified_email": True,
            "multipass_identifier": None,
            "tax_exempt": False,
            "tax_exemptions": {},
            "phone": "+15552345678",
            "tags": "loyal, returning",
            "currency": "USD",
            "addresses": {},
            "admin_graphql_api_id": "gid://shopify/Customer/567890123",
            "default_address": {}
          },
          "id": 2345678901,
          "order_number": 1005,
          "processed_at": "2023-05-15T12:45:00-07:00",
          "subtotal_price": "2599.99",
          "total_tax": "84.00",
          "total_tip_received": "10.25",
          "total_weight": 3500,
          "updated_at": "2023-05-15T14:30:00-07:00",
          "user_id": 890123456,
          "order_status_url": {
            "order_status_url": "https://checkout.shopify.com/112233/checkouts/2345678901qwertyuiop/token?key=zyxwvuts0987654321"
          }
        },
        "transaction_1": {
          "billing_address": {
            "address1": "789 Elmwood Rd",
            "address2": None,
            "city": "Seattle",
            "company": None,
            "country": "USA",
            "first_name": "David",
            "last_name": "Smith",
            "phone": "(555)987-6543",
            "province": "Washington",
            "zip": "98101",
            "name": "David Smith",
            "province_code": "WA",
            "country_code": "US",
            "latitude": "47.6062",
            "longitude": "-122.3321"
          },
          "confirmation_number": "POIUYTREW3",
          "currency": "USD",
          "customer": {
            "id": 345678901,
            "email": "david.smith@example.com",
            "accepts_marketing": False,
            "created_at": "2023-08-20T11:30:00-07:00",
            "updated_at": "2023-08-20T11:30:00-07:00",
            "first_name": "David",
            "last_name": "Smith",
            "state": "active",
            "note": None,
            "verified_email": True,
            "multipass_identifier": None,
            "tax_exempt": False,
            "tax_exemptions": {},
            "phone": "+15559876543",
            "tags": "loyal",
            "currency": "USD",
            "addresses": {},
            "admin_graphql_api_id": "gid://shopify/Customer/345678901",
            "default_address": {}
          },
          "id": 3456789012,
          "order_number": 1006,
          "processed_at": "2023-10-10T09:00:00-07:00",
          "subtotal_price": "2799.99",
          "total_tax": "78.00",
          "total_tip_received": "9.75",
          "total_weight": 3500,
          "updated_at": "2023-05-15T14:30:00-07:00",
          "user_id": 890123452,
          "order_status_url": {
            "order_status_url": "https://checkout.shopify.com/112233/checkouts/2345678901qwertyuiop/token?key=zyxwvuts0987654321"
          }
        }
      }
    }
  ]

Now we want to use Lume to map this source data automatically, using an existing pipeline. To do so,
1. Get the corresponding pipeline for your data.
2. Create a job for the pipeline, and provide the source data.
3. Run the job
4. Get output from the finished job

##### 1. Get the corresponding pipeline for your data

Depending on where your source data arrived (system x, api y, etc), use that knowledge to fetch the corresponding pipeline via the pipeline name, `ecomm_test` in this case. Pipeline names allow you use easily find pipelines, and customers typically use semantic text (e.g. systemA_pipeline_to_X_target) or hashes to discern this. 

In [13]:
pipeline_name = 'ecomm_test'
pipeline = await get_pipeline_with_name(pipeline_name)
pipeline


{'id': '4e11c124-b959-461f-97c3-8939eae317d5',
 'created_at': '2024-04-09T16:19:51.378888Z',
 'name': 'ecomm_test',
 'description': '',
 'target_schema': {'$schema': 'http://json-schema.org/draft-07/schema#',
  'type': 'object',
  'properties': {'product': {'type': 'object',
    'properties': {'product-make-model': {'type': 'string',
      'description': 'The make, model, and version of the product. Should be formatted by Make, Model, Version'},
     'body_html': {'type': 'string',
      'description': 'HTML content describing the product.'},
     'created_at': {'type': 'string',
      'format': 'date-time',
      'description': 'The date and time when the product was created.'},
     'handle': {'type': 'string',
      'description': 'A unique string identifier for the product.'},
     'id': {'type': 'integer',
      'description': 'The unique numeric identifier for the product.'},
     'currency': {'type': ['string', 'null'],
      'description': 'Choose a currency to assign to the pr

##### 2. Create a job for the pipeline, and provide the source data.

Use the id of the returned pipeline to create a job. Also pass in the source data to map.

In [14]:
job = await create_job(pipeline['id'], source_data)
job

{'id': 'bbeeed74-d253-4bb0-8caa-3b116dfed3d3',
 'created_at': '2024-04-18T14:49:35.370919Z',
 'pipeline_id': '4e11c124-b959-461f-97c3-8939eae317d5'}

##### 3. Run the job

In [15]:
result = await run_job(job['id'])
result

{'id': '2abaf374-da06-4a75-b177-24729e952587',
 'created_at': '2024-04-18T14:49:36.064387Z',
 'job_id': 'bbeeed74-d253-4bb0-8caa-3b116dfed3d3',
 'workshop_id': None,
 'pipeline_id': '4e11c124-b959-461f-97c3-8939eae317d5',
 'status': 'finished',
 'spec': {'product': {'product-make-model': {'@sources': ['product.model',
     'product.version',
     'product.make'],
    '@default_values': []},
   'body_html': {'@sources': ['product.body_html'], '@default_values': []},
   'created_at': {'@sources': ['product.created_at'], '@default_values': []},
   'handle': {'@sources': ['product.handle'], '@default_values': []},
   'id': {'@sources': ['product.id'], '@default_values': []},
   'currency': {'@sources': ['recent_transactions.transaction_1.currency'],
    '@default_values': [None]},
   'description': {'@sources': ['product.body_html'],
    '@default_values': [None]},
   'name': {'@sources': ['product.title'], '@default_values': [None]},
   'weight': {'@sources': ['product.id'], '@default_val

Once a job run executes to completion, a Result object is returned. The Result provides a few pieces of key information:
1. Status: the status of the job. If finished, the job executed with no errors or flags. 
2. spec: the high-order mapping logic and lookup table of the pipeline used on this job. 
3. id: the Result id. Use this to access the output mappings of the job run via Get Mappings For Result.

##### 4. Get output from the finished job

A finished job allows you to then access the spec and mapped data. First, let's access the spec via result. The spec indicates:
1. Which source properties are used to map to each target property
2. The default value is the source property is not present, or if there is no source property indicated.

In [16]:
spec = result['spec']
spec

{'product': {'product-make-model': {'@sources': ['product.model',
    'product.version',
    'product.make'],
   '@default_values': []},
  'body_html': {'@sources': ['product.body_html'], '@default_values': []},
  'created_at': {'@sources': ['product.created_at'], '@default_values': []},
  'handle': {'@sources': ['product.handle'], '@default_values': []},
  'id': {'@sources': ['product.id'], '@default_values': []},
  'currency': {'@sources': ['recent_transactions.transaction_1.currency'],
   '@default_values': [None]},
  'description': {'@sources': ['product.body_html'],
   '@default_values': [None]},
  'name': {'@sources': ['product.title'], '@default_values': [None]},
  'weight': {'@sources': ['product.id'], '@default_values': [None]},
  'sku': {'@sources': ['product.sku', 'product.variants'],
   '@default_values': [None]},
  'cost_price': {'@sources': ['product.variants.0.price'],
   '@default_values': [None]},
  'product_type': {'@sources': ['product.product_type'],
   '@default_va

Now, let's view the spec for the `product.name` target property.

In [17]:
product_name_spec = spec['product']['name']
product_name_spec

{'@sources': ['product.title'], '@default_values': [None]}

You can see that this mapping is doing a 1:1 extract from the `product.title` source property, and it is defaulting to `null` if the source property does not exist in the record.

Now, let's access the mapped data by querying the underlying mappings.

In [18]:
mappingsPage = await get_mappings_from_result(result['id'], page=1, size=10)
mappings = mappingsPage['items']
mappings

[{'index': 0,
  'source_record': {'product': {'make': 'iPod',
    'model': 'Nano',
    'version': 'V2.3',
    'body_html': "It's the small iPod with a big idea: Video.",
    'created_at': '2012-02-15T15:12:21-05:00',
    'handle': 'ipod-nano',
    'id': 632910392,
    'image': {'id': 850703190,
     'created_at': '2018-01-08T12:34:47-05:00',
     'updated_at': '2018-01-08T12:34:47-05:00',
     'width': 110,
     'height': 140,
     'x': 'http://example.com/burton.jpg'},
    'options': {'id': 594680422,
     'product_id': 632910392,
     'name': 'Color',
     'position': 1,
     'values': ['Pink', 'Red', 'Green', 'Black']},
    'product_type': 'Cult Products',
    'published_at': '2007-12-31T19:00:00-05:00',
    'published_scope': 'global',
    'status': 'active',
    'tags': 'Emotive, Flash Memory, MP3, Music',
    'template_suffix': 'special',
    'title': 'IPod Nano - 8GB',
    'updated_at': '2012-08-24T14:01:47-04:00',
    'variants': [{'barcode': '1234_pink',
      'compare_at_pric

Now, the source data was successfully mapped by Lume to your desired target schema, and you have access to the data via `mappings`. For each source record, you can access the corresponding mapped data via the `index` and `mapped_data` properties. `mappings` is a list where each object of mapped data represents a source record.

For more detail, we can zoom in to the first record of `mappings`. Each object will have information on:
1. `index`: The index of the source record in question
2. `source_record`
2. `mapped_data`: The final mapped data record
3. `message`: either `success` or a json error. If a record is not marked as `success`, it failed a Lume type check against your specified target schema.

In [19]:
mappings[0]

{'index': 0,
 'source_record': {'product': {'make': 'iPod',
   'model': 'Nano',
   'version': 'V2.3',
   'body_html': "It's the small iPod with a big idea: Video.",
   'created_at': '2012-02-15T15:12:21-05:00',
   'handle': 'ipod-nano',
   'id': 632910392,
   'image': {'id': 850703190,
    'created_at': '2018-01-08T12:34:47-05:00',
    'updated_at': '2018-01-08T12:34:47-05:00',
    'width': 110,
    'height': 140,
    'x': 'http://example.com/burton.jpg'},
   'options': {'id': 594680422,
    'product_id': 632910392,
    'name': 'Color',
    'position': 1,
    'values': ['Pink', 'Red', 'Green', 'Black']},
   'product_type': 'Cult Products',
   'published_at': '2007-12-31T19:00:00-05:00',
   'published_scope': 'global',
   'status': 'active',
   'tags': 'Emotive, Flash Memory, MP3, Music',
   'template_suffix': 'special',
   'title': 'IPod Nano - 8GB',
   'updated_at': '2012-08-24T14:01:47-04:00',
   'variants': [{'barcode': '1234_pink',
     'compare_at_price': None,
     'created_at': 

##### 5. Pipe the output to your end destination
Iterate through the mappings to get the final mapped data, and send it to the next step of your workflow. Use the `get_all_mappings` utility function to iterate through all the pages of data, if needed.

In [20]:
all_mappings = await get_all_mappings(result['id'])
all_mappings

# TODO: send all_mappings to the next step in the pipeline

[{'index': 0,
  'source_record': {'product': {'make': 'iPod',
    'model': 'Nano',
    'version': 'V2.3',
    'body_html': "It's the small iPod with a big idea: Video.",
    'created_at': '2012-02-15T15:12:21-05:00',
    'handle': 'ipod-nano',
    'id': 632910392,
    'image': {'id': 850703190,
     'created_at': '2018-01-08T12:34:47-05:00',
     'updated_at': '2018-01-08T12:34:47-05:00',
     'width': 110,
     'height': 140,
     'x': 'http://example.com/burton.jpg'},
    'options': {'id': 594680422,
     'product_id': 632910392,
     'name': 'Color',
     'position': 1,
     'values': ['Pink', 'Red', 'Green', 'Black']},
    'product_type': 'Cult Products',
    'published_at': '2007-12-31T19:00:00-05:00',
    'published_scope': 'global',
    'status': 'active',
    'tags': 'Emotive, Flash Memory, MP3, Music',
    'template_suffix': 'special',
    'title': 'IPod Nano - 8GB',
    'updated_at': '2012-08-24T14:01:47-04:00',
    'variants': [{'barcode': '1234_pink',
      'compare_at_pric